In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.losses import MeanSquaredError
from keras.optimizers import Adam

In [2]:
DATA_PATH = 'data_cars/'
all_files = os.listdir(DATA_PATH)

all_dataframes = []
for index, file in enumerate(all_files):
    print(f"Reading file: {file}")
    file_name = file.split('.')[0]
    df = pd.read_csv(DATA_PATH + file, sep=';')

    df['date'] = pd.to_datetime(df[file_name], format='%Y-%m-%d %H:%M')
    df = df.drop(columns=[file_name])

    df = df.set_index('date')
    df.columns = [f"{file_name}_{col}" for col in df.columns if col != 'date']
    all_dataframes.append(df)
    print(f"Finished reading file: {file}, shape = {df.shape}")

combined_df = pd.concat(all_dataframes, axis=1)
combined_df.fillna(method='ffill', inplace=True)
combined_df['hour'] = combined_df.index.hour
combined_df['day_of_week'] = combined_df.index.dayofweek

data = np.array(combined_df, dtype=float)[:, :-2]

Reading file: K120.csv
Finished reading file: K120.csv, shape = (2880, 7)
Reading file: K134.csv
Finished reading file: K134.csv, shape = (2880, 7)
Reading file: K140.csv
Finished reading file: K140.csv, shape = (2880, 5)
Reading file: K159.csv
Finished reading file: K159.csv, shape = (2880, 11)
Reading file: K405.csv
Finished reading file: K405.csv, shape = (2880, 19)
Reading file: K406.csv
Finished reading file: K406.csv, shape = (2880, 8)
Reading file: K701.csv
Finished reading file: K701.csv, shape = (2880, 7)
Reading file: K702.csv
Finished reading file: K702.csv, shape = (2880, 8)
Reading file: K703.csv
Finished reading file: K703.csv, shape = (2880, 10)
Reading file: K709.csv
Finished reading file: K709.csv, shape = (2880, 17)
Reading file: K711.csv
Finished reading file: K711.csv, shape = (2880, 31)


In [3]:
def splitSequence(seq, n_steps):

    #Declare X and y as empty list
    X = []
    y = []

    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps

        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break

        # Create input and output sequence
        # Last 2 columns are time of day and day of week
        seq_X, seq_y = seq[i:lastIndex], seq[lastIndex]

        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)

    return X,y

In [4]:
def shiftSequence(seq, n_steps):

    #Declare X and y as empty list
    X = []
    y = []

    for i in range(len(seq)):
        #get the last index
        lastIndex = i + n_steps

        #if lastIndex is greater than length of sequence then break
        if lastIndex > len(seq) - 1:
            break

        # Create input and output sequence
        # Last 2 columns are time of day and day of week
        seq_X, seq_y = seq[i:lastIndex], seq[i+1:lastIndex+1]

        #append seq_X, seq_y in X and y list
        X.append(seq_X)
        y.append(seq_y)
        #Convert X and y into numpy array
    X = np.array(X)
    y = np.array(y)

    return X,y

In [5]:
num_of_steps = data.shape[0]
train_size = 0.6
val_size = 0.15
shuffle = False
look_back = 80

x, y = splitSequence(data, look_back)

x_int, y_int = splitSequence(data[:, :7], look_back)

if shuffle:
    idx = np.random.permutation(len(x))
    x,y = x[idx], y[idx]

num_train = int(num_of_steps * train_size)
num_val = int(num_of_steps * val_size)

x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = x[num_train:num_train + num_val], y[num_train:num_train + num_val]
x_test, y_test = x[num_train + num_val:], y[num_train + num_val:]

x_train_singleInt, y_train_singleInt = x_int[:num_train], y_int[:num_train]
x_val_singleInt, y_val_singleInt = x_int[num_train:num_train + num_val], y_int[num_train:num_train + num_val]
x_test_singleInt, y_test_singleInt = x_int[num_train + num_val:], y_int[num_train + num_val:]


(1728, 80)
(1728, 80)


In [10]:
x_one, y_one = shiftSequence(data[:, 0], look_back)
print(x_one.shape, y_one.shape)
x_one[0], y_one[0]

x_train_single, y_train_single = x_one[:num_train], y_one[:num_train]
x_val_single, y_val_single = x_one[num_train:num_train + num_val], y_one[num_train:num_train + num_val]
x_test_single, y_test_single = x_one[num_train + num_val:], y_one[num_train + num_val:]

print(y_train_single.shape)
print(x_train_single.shape)

(2800, 80) (2800, 80)


(array([ 3.,  4.,  0.,  0.,  1.,  4.,  4.,  1.,  0.,  1.,  1.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  1.,
         4.,  5.,  6.,  7., 13., 21., 18., 43., 42., 26., 20., 17., 18.,
        21., 31., 16., 24., 20., 30., 21., 22., 69., 38., 26., 41., 25.,
        25., 24., 33., 33., 32., 34., 63., 61., 48., 55., 34., 39., 28.,
        36., 52., 30., 35., 62., 37., 25., 15., 20., 17., 16.,  7.,  8.,
        16., 11.]),
 array([ 4.,  0.,  0.,  1.,  4.,  4.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  2.,  4.,  1.,  4.,
         5.,  6.,  7., 13., 21., 18., 43., 42., 26., 20., 17., 18., 21.,
        31., 16., 24., 20., 30., 21., 22., 69., 38., 26., 41., 25., 25.,
        24., 33., 33., 32., 34., 63., 61., 48., 55., 34., 39., 28., 36.,
        52., 30., 35., 62., 37., 25., 15., 20., 17., 16.,  7.,  8., 16.,
        11., 12.]))

In [60]:
print(x_train_single[0, :3])
print(y_train_single[0, :3])

[[-0.75016484]
 [-0.66650333]
 [-1.00114935]]
[0.00278872]


In [25]:
from keras import layers

#Following structure layed out in github.com/locuslab/TCN
def createTemporalConvNetwork(n_layers, n_sensors, look_back, n_outputs, kernel_size=2, dropout=0.2):
    modelTCN = keras.models.Sequential()
    modelTCN.add(layers.InputLayer((look_back, n_sensors)))
    for i in range(n_layers):
        modelTCN.add(layers.Conv1D(128, kernel_size=kernel_size, padding='causal', activation='relu', dilation_rate=2**i))
        modelTCN.add(layers.Dropout(dropout))
        modelTCN.add(layers.Conv1D(128, kernel_size=kernel_size, padding='causal', activation='relu', dilation_rate=2**i))
        modelTCN.add(layers.Dropout(dropout))
    modelTCN.add(layers.Conv1D(n_outputs, kernel_size=kernel_size, padding='causal', activation='relu', dilation_rate=2**(i+1)))
    return modelTCN

In [27]:
modelCNN = createTemporalConvNetwork(4, 1, look_back, 1, kernel_size=2, dropout=0.3)

modelCNN.summary()

modelCNN.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.001),
    metrics=[keras.metrics.RootMeanSquaredError()],
)

modelCNN.fit(
    x=x_train_single,
    y=y_train_single,
    validation_data=(x_val_single, y_val_single),
    epochs=500,
    batch_size=32,
    #makes the training stop early if it notices no improvements on the validation set 10 times in a row, to prevent overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=5)],
)

# make predictions
trainPredict = modelCNN.predict(x_train_single)
testPredict = modelCNN.predict(x_test_single)

# calculate root mean squared error
trainScore = np.sqrt((mean_squared_error(y_train_single[:, -1], trainPredict[:, -1])))
print(f'Train Score: {trainScore:.2f} RMSE')
testScore = np.sqrt(mean_squared_error(y_test_single[:, -1], testPredict[:, -1]))
print(f'Test Score: {testScore:.2f} RMSE')

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_107 (Conv1D)         (None, 80, 128)           384       
                                                                 
 dropout_96 (Dropout)        (None, 80, 128)           0         
                                                                 
 conv1d_108 (Conv1D)         (None, 80, 128)           32896     
                                                                 
 dropout_97 (Dropout)        (None, 80, 128)           0         
                                                                 
 conv1d_109 (Conv1D)         (None, 80, 128)           32896     
                                                                 
 dropout_98 (Dropout)        (None, 80, 128)           0         
                                                                 
 conv1d_110 (Conv1D)         (None, 80, 128)         

In [ ]:
plt.plot(testY, label='actual')
plt.plot(testPredict, label='predict')
plt.legend()

In [ ]:
modelCNN = createTemporalConvNetwork(4, 7, look_back, 1, kernel_size=2)

#modelCNN.summary()

modelCNN.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.0001),
    metrics=[keras.metrics.RootMeanSquaredError()],
)

history = modelCNN.fit(
    x=x_train_singleInt,
    y=y_train_single,
    validation_data=(x_val_singleInt, y_val_single),
    epochs=500,
    batch_size=32,
    #makes the training stop early if it notices no improvements on the validation set 10 times in a row, to prevent overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=8)],
)

# make predictions
trainPredict = modelCNN.predict(x_train_singleInt)
testPredict = modelCNN.predict(x_test_singleInt)
#invert predictions
trainPredict = singleScaler.inverse_transform(trainPredict)
trainY = singleScaler.inverse_transform(y_train_single)
testPredict = singleScaler.inverse_transform(testPredict)
testY = singleScaler.inverse_transform(y_test_single)

# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY , trainPredict))
print(f'Train Score: {trainScore:.2f} RMSE')
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Test Score: {testScore:.2f} RMSE')

In [ ]:
modelCNN = createTemporalConvNetwork(8, 130, look_back, 130, kernel_size=3)

#modelCNN.summary()

modelCNN.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=0.0001),
    metrics=[keras.metrics.RootMeanSquaredError()],
)

history = modelCNN.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val, y_val),
    epochs=500,
    batch_size=32,
    #makes the training stop early if it notices no improvements on the validation set 10 times in a row, to prevent overfitting
    callbacks=[keras.callbacks.EarlyStopping(patience=5)],
)

# make predictions
trainPredict = modelCNN.predict(x_train)
testPredict = modelCNN.predict(x_test)
#invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(y_train)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(y_test)

# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY , trainPredict))
print(f'Train Score: {trainScore:.2f} RMSE')
testScore = np.sqrt(mean_squared_error(testY, testPredict))
print(f'Test Score: {testScore:.2f} RMSE')

In [ ]:
plt.plot(testY[:,0], label='actual')
plt.plot(testPredict[:,0], label='predict')
plt.legend()

In [ ]:
def getMostCorrelated(sensor, no_extraSensors, data):
    corr = np.corrcoef(data.T)
    best = np.argsort(corr[sensor])[-no_extraSensors:]
    return data[:, best]

def getScore(model, x, y, scaler):
    predicts = model.predict(x)
    scaled_predict = scaler.inverse_transform(predicts)
    return np.sqrt(mean_squared_error(y, scaled_predict))
    
chosenSensor, no_inputSensors = 0, 10
mostCorrelated = getMostCorrelated(chosenSensor, no_inputSensors, data)

num_of_steps = data.shape[0] - look_back
train_size = 0.6
val_size = 0.15
num_train = int(num_of_steps * train_size)
num_val = int(num_of_steps * val_size)

def experiment(no_inputs, no_samples, averaging):
    sample_scores = np.zeros((no_samples, len(no_inputs)))
    for sample in range(no_samples):
        predicted_sensor = np.random.choice(np.arange(data.shape[1]))
        _, y = data[:, predicted_sensor]
        y_scaler = StandardScaler()
        y = y_scaler.fit_transform(y)
        y_train, y_val, y_test = y[:num_train], y[num_train:num_val], y[num_train+num_val:]

        for idx, inputs in enumerate(no_inputs):
            correlated_data = getMostCorrelated(predicted_sensor, inputs, data)
            scaler = StandardScaler()
            correlated_data = scaler.fit_transform(correlated_data)
            x, _ = splitSequence(correlated_data)
            x_train, x_val, x_test = x[:num_train], x[num_train:num_val], x[num_train+num_val:]
            
            scores = np.zeros(averaging)
            for av in range(averaging):
                model = createTemporalConvNetwork(4, inputs, look_back, 1)
                cb = [keras.callbacks.EarlyStopping(patience=10), keras.callbacks.ModelCheckpoint('./checkpoints/', save_best_only=True)]
                model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=32, epochs=500, callbacks=cb)
                scores[av] = getScore(model, x_test, y_test, y_scaler)
            sample_scores[sample, idx] = np.average(scores)
    final_scores = np.average(sample_scores, axis=0)
    plt.plot(final_scores, no_inputs)
    plt.show()
